In [ ]:
import ee
import geemap
import geemap.chart as chart
import numpy as np
import pandas as pd
import copy
import math
import sys,os
sys.path.append(os.path.dirname(os.path.abspath('__file__')))
from tqdm import trange,tqdm
from Basic_tools import *
from New_Correct import *
from Correct_filter import *
from S2_filter import *
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

# 数据加载,与全局变量初始化

In [ ]:
#--------------------------预加载冰湖数据
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

#--------------------确定时间范围
# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
START_DATE  = ee.Date('2019-07-01')
END_DATE   = ee.Date('2019-08-30')
S2_START_DATE = ee.Date('2019-06-01')
S2_END_DATE  = ee.Date('2019-09-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')

#-------------------DEM选择
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)

#-------------------几何畸变参数
models = ['volume', 'surface', None]     # 地形矫正模型
Model = models[0]
Origin_scale = 10
projScale = 30

#-------------------S2 clodless参数
CLOUD_FILTER = 60         # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15        # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15       # 非水暗像素判断阈值
CLD_PRJ_DIST = 1         # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50            # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input

# 测试

## 选择一个冰湖数据

In [ ]:
# 选择一个点做实验
i=1 #394(面积大) #53 #11(影像都用nodata) #149(坐标系问题) #1143(畸变) #539(中等交并比)  #1296(高等交并比)

AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

# 计算面积
AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()
# 缩小
if AOI_area_area < 1:
    AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
else:
    AOI_area_buffer = AOI_area.buffer(distance=-400)

# 扩大包络矩形AOI,保证背景像素占比最大，原有基础上相应+100
if AOI_area_area < 1:
    AOI_buffer = AOI.buffer(distance=400).bounds()
else:
    AOI_buffer = AOI.buffer(distance=500).bounds()

# 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
s1_ascending, s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                                                            Filter='RefinedLee',FilterSize=30)

# 采用S2cloudless生产一张合成的无云图像
s2_sr_median = merge_s2_collection(AOI_buffer,START_DATE,END_DATE,
                      CLOUD_FILTER,BUFFER,CLD_PRJ_DIST,CLD_PRB_THRESH,NIR_DRK_THRESH)

## 函数：计算几何畸变区域，然后采用ImageCollection进行融合

In [ ]:
def CalDitorAndCombin(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],):
  # 当'synthesis': 1，则跳过几何畸变校正步骤，直接进入识别
  synthesis_a  = ee.Image(s1_ascending).get('synthesis').getInfo()
  synthesis_d  = ee.Image(s1_descending).get('synthesis').getInfo()
  
  if synthesis_a or synthesis_d:
      print('Image with synthesis')
      s1_ascending = s1_ascending.rename(['VV_gamma0_flatDB','VH_gamma0_flatDB','incAngle'])
      s1_descending = s1_descending.rename(['VV_gamma0_flatDB','VH_gamma0_flatDB','incAngle'])
      default_prj = s1_ascending.select(0).projection()
      Combin_ad = ee.ImageCollection([s1_ascending,s1_descending])
      s1_unit_mean_ = Combin_ad.mean().reproject(default_prj)
      s1_unit_max_  = Combin_ad.max().reproject(default_prj)
      s1_unit_min_  = Combin_ad.min().reproject(default_prj)
      volumetric_dict = {}
      volumetric_dict['ASCENDING'] = s1_ascending
      volumetric_dict['ASCENDING_parms'] = {'proj':default_prj}
      volumetric_dict['DESCENDING'] = s1_descending
      volumetric_dict['DESCENDING_parms'] = {'proj':s1_descending.select(0).projection()}
      synthesis = 1
      return volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_,synthesis
  else:
      print('Image with slop correction and volumetric')
      synthesis = 0
      volumetric_dict = my_slope_correction(s1_ascending,s1_descending,AOI_buffer,DEMNASA,Model,Origin_scale)

  # ----------------------Z-scor均衡化，直方图匹配，在局部进行直方图匹配
  Bands_ = Bands_
  Ascending_Img = volumetric_dict['ASCENDING'].select(Bands_)
  Descending_Img = volumetric_dict['DESCENDING'].select(Bands_)

  ASCENDING_meanStd = meanStd_norm(Ascending_Img,Bands_,scale=Origin_scale)
  DESSCENDING_meanStd = meanStd_norm(Descending_Img,Bands_,scale=Origin_scale)

  # 经过z-score，无需再进行均值迁移
  Match_Ascending = histogramMatching(ASCENDING_meanStd, DESSCENDING_meanStd
                      ,AOI_buffer,Bands_,Bands_
                      ,Histscale=projScale,maxBuckets=1024).clip(AOI_buffer)

  volumetric_dict['ASCENDING'] = replaceBands(volumetric_dict['ASCENDING'],Match_Ascending)
  volumetric_dict['DESCENDING'] = replaceBands(volumetric_dict['DESCENDING'],DESSCENDING_meanStd)


  # -----------------------基于线性关系。检测几何畸变
  Templist_A = AuxiliaryLine2Point(volumetric_dict['ASCENDING'],volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                                volumetric_dict['ASCENDING_parms']['Auxiliarylines'],
                                projScale)
  LeftLayoverA,RightLayoverA,ShadowA = Line_Correct(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale)

  Templist_D = AuxiliaryLine2Point(volumetric_dict['DESCENDING'],volumetric_dict['DESCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['DESCENDING_parms']['coordinates_dict'],
                                volumetric_dict['DESCENDING_parms']['Auxiliarylines'],
                                projScale)
  LeftLayoverD,RightLayoverD,ShadowD =  Line_Correct(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale)


  def Cal_mask(LeftLayover,RightLayover,Shadow,AOI_buffer):
      # 判断是否为空
      left_empty = LeftLayover.bandNames().length().eq(0)
      right_empty = RightLayover.bandNames().length().eq(0)
      shadow_empty = Shadow.bandNames().length().eq(0)

      # 只合并非空图像
      result = ee.Image(ee.Algorithms.If(left_empty, ee.Image(), LeftLayover))
      result = ee.Image(ee.Algorithms.If(right_empty,result,result.Or(RightLayover)))
      result = ee.Image(ee.Algorithms.If(shadow_empty,result,result.Or(Shadow)))
      return result.clip(AOI_buffer)

  LeftLayoverA,RightLayoverA,ShadowA = LeftLayoverA.mask(),RightLayoverA.mask(),ShadowA.mask()
  LeftLayoverD,RightLayoverD,ShadowD = LeftLayoverD.mask(),RightLayoverD.mask(),ShadowD.mask()

  A_mask_ = Cal_mask(LeftLayoverA,RightLayoverA,ShadowA,AOI_buffer)
  D_mask_ = Cal_mask(LeftLayoverD,RightLayoverD,ShadowD,AOI_buffer)
  All_distor = A_mask_.And(D_mask_)

  A_empty = A_mask_.bandNames().contains('constant')
  A_empty = ee.Number(ee.Algorithms.If(A_empty, 1, 0))
  D_empty = D_mask_.bandNames().contains('constant')
  D_empty = ee.Number(ee.Algorithms.If(D_empty, 1, 0))

  s1_ascending  = volumetric_dict['ASCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])
  s1_descending = volumetric_dict['DESCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])

  s1_ascending_ = ee.Image(ee.Algorithms.If(A_empty, ee.Image(), s1_ascending.where(A_mask_,s1_descending)))
  s1_descending_ = ee.Image(ee.Algorithms.If(D_empty, ee.Image(), s1_descending.where(D_mask_,s1_ascending)))
  Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])

  s1_unit_mean_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                s1_ascending.add(s1_descending).divide(2),
                Combin_AD.mean())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

  s1_unit_max_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                s1_ascending.max(s1_descending),
                Combin_AD.max())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

  s1_unit_min_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                s1_ascending.min(s1_descending),
                Combin_AD.min())).reproject(volumetric_dict['ASCENDING_parms']['proj'])
  
  return volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_,synthesis

# volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_,synthesis = \
#             CalDitorAndCombin(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'])

## 展示一下数据融合效果

In [ ]:
# Map = geemap.Map() # basemap='HYBRID'
# Map.centerObject(AOI_buffer, zoom=15)
# # Map.addLayer(volumetric_dict['ASCENDING'].select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb')
# # Map.addLayer(LeftLayover.randomVisualizer(),{},'LeftLayover')
# # Map.addLayer(RightLayover.randomVisualizer(),{},'RightLayover')
# # Map.addLayer(Shadow.randomVisualizer(),{},'Shadow')
# Map.addLayer(s2_sr_median,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000, 'gamma': 2.0},'s2_sr_median')
# # Map.addLayer(A_mask_.randomVisualizer(),{},'A_mask')
# # Map.addLayer(D_mask_.randomVisualizer(),{},'D_mask')
# # Map.addLayer(All_distor.randomVisualizer().clip(AOI_buffer), {}, 'All_distor')
# Map.addLayer(volumetric_dict['ASCENDING'].select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb_A')
# Map.addLayer(volumetric_dict['DESCENDING'].select('red','green','blue'), {'min':0,'max':255}, 'no_data_maskrgb_D')
#
# Map.addLayer(volumetric_dict['ASCENDING'].select("VV_sigma0"),{'min':-18,'max':5},'ascending')
# Map.addLayer(volumetric_dict['DESCENDING'].select("VV_sigma0"),{'min':-18,'max':5},'descending')
#
# Map.addLayer(volumetric_dict['ASCENDING'].select("VV_gamma0_flatDB"),{'min':-2,'max':2},'ascending_flat')
# Map.addLayer(volumetric_dict['DESCENDING'].select("VV_gamma0_flatDB"),{'min':-2,'max':2},'descending_flat')
#
# Map.addLayer(s1_unit_mean_.select("VV_gamma0_flatDB"),{'min':-2,'max':2},'s1_unit_mean_')
# Map.addLayer(s1_unit_max_.select("VV_gamma0_flatDB"),{'min':-2,'max':2},'s1_unit_max_')
# Map.addLayer(s1_unit_min_.select("VV_gamma0_flatDB"),{'min':-2,'max':2},'s1_unit_min_')
# Map

## 函数：提取冰湖，检测效果

### 聚类方式提取冰湖

In [ ]:
from Extract_algorithm import Cluster_extract as Cluster

# MeanStd归一化，保留中间90%,以-2-2近似替代
def Percent90(Image:ee.Image):
    Image = Image.where(Image.gte(2),2)
    Image = Image.where(Image.lte(-2),-2)
    return Image

def Cluster_math(method: str, img, bands: list, index: str, visual: bool, save: bool, region=None,proj=None,Synth=True):
    '''method ('Kmean','SNIC','SNIC_Kmean','LVQ','Xmeans','Cobweb','CascadeKMeans')'''
    img = img.select(bands)
    if not Synth:
        print('90_percent')
        img = Percent90(img).clip(region)
    img = minmax_norm(img,bands,scale=Origin_scale)
    img = img.unmask(1).clip(region)
    
    if proj:
      img = img.reproject(proj)
        
    if method == 'Kmean':
        result = Cluster.afn_Kmeans(img, region)
    elif method == 'Cobweb':
        result = Cluster.afn_Cobweb(img, region)
    elif method == 'Xmeans':
        result = Cluster.afn_Xmeans(img, region)
    elif method == 'LVQ':
        result = Cluster.afn_LVQ(img, region)
    elif method == 'CascadeKMeans':
        result = Cluster.afn_CascadeKMeans(img, region)
    elif method == 'SNIC':
        result = Cluster.afn_SNIC(img)
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(result.select(0).projection().getInfo()['crs'], None, 10)
    elif method == 'SNIC_Kmean':
        result = Cluster.afn_SNIC(img)
        # 默认舍弃cluster和seed
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(result.select(0).projection().getInfo()['crs'], None, 10)
        result0 = Cluster.afn_Kmeans(result, region)  # 原始图像不参与
        result1 = Cluster.afn_Kmeans(result.addBands(img), region)  # 原始图像参与    .unmask(10)

    if visual:
        Map = geemap.Map(basemap='HYBRID')  #
        Map.centerObject(AOI_point, zoom=15)
        Map.addLayer(img, {'min': 0.2, 'max': 0.8}, 'Origin')
        if method in ['Kmean', 'Cobweb', 'Xmeans', 'LVQ', 'CascadeKMeans']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC':
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC_Kmean':
            Map.addLayer(result0.randomVisualizer(), {}, 'SNIC_Kmean_NoOrigin')
            Map.addLayer(result1.randomVisualizer(), {}, 'SNIC_Kmean_YesOrigin')
        else:
            print('Please check your method str')
    else:
        Map = None
    if save:
        if method == 'SNIC_Kmean':
            Geemap_export(filename=index + 'NoOrigin' + method + '.tif', collection=False, image=result0,
                          region=region, scale=10)
            Geemap_export(filename=index + 'YesOrigin' + method + '.tif', collection=False, image=result1,
                          region=region, scale=10)
        else:
            Geemap_export(filename=index + method + '.tif', collection=False, image=result, region=region, scale=10)
        pass
    if method == 'SNIC_Kmean':
        return Map, result0.addBands(result1)
    else:
        return Map, result

# Map,result=Cluster_math(method='Kmean',img=s1_unit_mean_,bands=['VV_gamma0_flatDB', 'VH_gamma0_flatDB'],
#                         index='',visual=True,save=False,region=AOI_buffer,proj=volumetric_dict['ASCENDING_parms']['proj'],Synth=synthesis)  # 如果迭代多个影像可以采用调整index
# Map

### 自适应阈值方式提取冰湖

In [ ]:
from Extract_algorithm import Adaptive_threshold as Adap

def Bandmath(method:str,img,band,index:str,visual:bool,save:bool,region=None,proj=None,Synth=True):
    '''
    method = ['otsu','histPeak']
    img: 仅单波段图像
    '''
    img = img.select(band)
#     assert len(img.bandNames().getInfo()) == 1 , '图像波段数必须唯一'
    if not Synth:
        print('90_percent')
        img = Percent90(img).clip(region)
    img = minmax_norm(img,band,scale=Origin_scale)
    img = img.unmask(1)
    if proj:
      img = img.reproject(proj)

    if method == 'otsu':
        histogram = get_histogram(img,region=region,scale=Origin_scale)
        Threshould_value = Adap.afn_otsu(histogram)
        result = img.select(0).gt(Threshould_value).clip(region)  #
        print('Threshould value is {}'.format(Threshould_value.getInfo()))

    elif method == 'histPeak':
        Threshould_value = Adap.afn_histPeak(img,region=region)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))

    '''还有很多自适应阈值方法
    li\yen\......使用try_all_threshold可以测试
    from skimage.filters import try_all_threshold,threshold_sauvola

    # try_all_threshold(img_numpy, figsize=(8, 5), verbose=False)
    test = threshold_sauvola(img_numpy, window_size=5, k=0.2, r=None)
    test2 = img_numpy>test
    '''

    if visual:
        Map = geemap.Map(basemap='HYBRID') #
        Map.centerObject(AOI_point, zoom=15)
        if method in ['otsu','histPeak']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        else:
            print('Wrong visual! Please check your method str')
    else:
      Map = None

    if save:
        if method in ['otsu','histPeak']:
            Geemap_export(filename=index+method+'.tif',collection=False,
                          image=result,region=region,scale=10)
        else:
            print('Wrong save! Please check your method str')

    return Map,result

# Map,result = Bandmath(method='otsu',img=s1_unit_mean_,band=['VV_gamma0_flatDB'],index='',
#                       visual=True,save=False,region=AOI_buffer,proj=volumetric_dict['ASCENDING_parms']['proj'],Synth=synthesis)
# Map

# 输出图像

In [ ]:
from Extract_algorithm import Reprocess,save_parms
os.chdir('./Test')
Methods = ('Kmean','SNIC_Kmean','LVQ','Xmeans','CascadeKMeans','otsu','histPeak')
resultbands = (0,1)
# Bands = ([0],[1],[2],[3],[0,1],[0,2],[0,3],[1,2],[1,3],[2,3],[0,1,2],[0,1,3],[1,2,3],[0,1,2,3])

Bands = (['VV_gamma0_flatDB'],['VH_gamma0_flatDB'],['VV_gamma0_flatDB','VH_gamma0_flatDB'])

mode = 'gpd'
logname = 'preSNIC_Kmean.csv'
shpname = 'preSNIC_Kmean.shp'

In [ ]:
i= 11
AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

# 计算面积
AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()

# 缩小冰湖面积
if AOI_area_area < 1:
    AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
else:
    AOI_area_buffer = AOI_area.buffer(distance=-400)

# 扩大包络矩形AOI,保证背景像素占比最大，原有基础上相应+100
if AOI_area_area < 1:
    AOI_buffer = AOI.buffer(distance=400).bounds()
else:
    AOI_buffer = AOI.buffer(distance=500).bounds()

# 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
s1_ascending, s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                              Filter='RefinedLee',FilterSize=30)

# 采用S2cloudless生产一张合成的无云图像
s2_sr_median = merge_s2_collection(AOI_buffer,START_DATE,END_DATE,
                      CLOUD_FILTER,BUFFER,CLD_PRJ_DIST,CLD_PRB_THRESH,NIR_DRK_THRESH)

volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_,synthesis = \
        CalDitorAndCombin(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'])

K = [1 if Method == 'SNIC_Kmean' else 0 for Method in Methods ]

# 检测是否内缩为空，否则用AOI_area替代
if AOI_area_buffer.coordinates().getInfo() == []:
    FilterBound = AOI_area
    print('AOI_area instead AOI_area_buffer')
else:
    FilterBound = AOI_area_buffer

for k,Method in zip(K,Methods):
    resultband = resultbands[k]
    for Band in Bands:
        if Method in ['Kmean','SNIC_Kmean','LVQ','Xmeans','CascadeKMeans']:
          Map,result = Cluster_math(method=Method,img=s1_unit_min_,bands=Band,index=''
                          ,visual=False,save=False,region=AOI_buffer,
                        proj=volumetric_dict['ASCENDING_parms']['proj'],Synth=synthesis)
          print('Method 1 {} Band={}'.format(Method,Band))
        elif Method in ['otsu','histPeak']:
          if len(Band)==1:
            Map,result = Bandmath(method=Method,img=s1_unit_min_,band=Band,index='',
                      visual=False,save=False,region=AOI_buffer,
                      proj=volumetric_dict['ASCENDING_parms']['proj'],Synth=synthesis)
            print('Method 2 {} Band={}'.format(Method,Band))
          else:
            continue
        else:
          print('Wrong method input!')
# 分类图转矢量
Union_ex = Reprocess.image2vector(result,
            resultband=resultband,
            GLarea=AOI_area_area,
            FilterBound=FilterBound)

if s1_ascending.get('synthesis').getInfo() == 0:
  a_name,a_date,a_nodata = ee.List([s1_ascending.get('system:index'),
                     s1_ascending.date().format('YYYY-MM-dd'),
                     s1_ascending.get('numNodata')]).getInfo()
else:
    a_name = 'None'; a_date = 'None' ; a_nodata='None'

if s1_descending.get('synthesis').getInfo() == 0:
    d_name,d_date,d_nodata = ee.List([s1_descending.get('system:index'),
                    s1_descending.date().format('YYYY-MM-dd'),
                    s1_descending.get('numNodata')]).getInfo()
else:
    d_name = 'None'; d_date = 'None' ; d_nodata='None'

pd_dict = {'a_name':a_name,'d_name':d_name,
        'a_date':a_date,'d_date':d_date,
        'a_nodata':a_nodata,'d_nodata':d_nodata}

In [ ]:
method=Method;img=s1_unit_min_;bands=Band;index=''
visual=False;save=False;region=AOI_buffer
proj=volumetric_dict['ASCENDING_parms']['proj'];Synth=synthesis

In [ ]:
img = img.select(bands)
if not Synth:
    print('90_percent')
    img = Percent90(img).clip(region)

In [ ]:
Image,Bands,scale = img,bands,Origin_scale
# proj = Image.projection()

In [ ]:
Image.projection().getInfo()

In [ ]:
def minmax_norm(Image:ee.Image,Bands,scale:int=10):
    proj = Image.projection()
    for i,eachName in enumerate(Bands):
        cal_band = Image.select(eachName)
        minmax = get_minmax(cal_band, scale=scale)
        nominator = cal_band.subtract(ee.Number(minmax.get('min')))
        denominator = ee.Number(minmax.get('max')).subtract(ee.Number(minmax.get('min')))
        if i == 0:
            result = nominator.divide(denominator).reproject(proj)
        else:
            result = result.addBands(nominator.divide(denominator)).reproject(proj)
    return result

In [ ]:
img = minmax_norm(img,bands,scale=Origin_scale)
img

In [ ]:
img = minmax_norm(img,bands,scale=Origin_scale)
img = img.unmask(1)
img = img.reproject(proj).clip(region)
img.projection().getInfo()

In [ ]:
for i in trange(Num_list):
    AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
    AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
    AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

    # 计算面积
    AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()

    # 缩小冰湖面积
    if AOI_area_area < 1:
        AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
    else:
        AOI_area_buffer = AOI_area.buffer(distance=-400)

    # 扩大包络矩形AOI,保证背景像素占比最大，原有基础上相应+100
    if AOI_area_area < 1:
        AOI_buffer = AOI.buffer(distance=400).bounds()
    else:
        AOI_buffer = AOI.buffer(distance=500).bounds()

    # 检测是否内缩为空，否则用AOI_area替代
    if AOI_area_buffer.coordinates().getInfo() == []:
        FilterBound = AOI_area
    else:
        FilterBound = AOI_area_buffer

    # 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
    s1_ascending, s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                                  Filter='RefinedLee',FilterSize=30)

    # 采用S2cloudless生产一张合成的无云图像
    s2_sr_median = merge_s2_collection(AOI_buffer,START_DATE,END_DATE,
                          CLOUD_FILTER,BUFFER,CLD_PRJ_DIST,CLD_PRB_THRESH,NIR_DRK_THRESH)

    volumetric_dict,s1_unit_mean_,s1_unit_max_,s1_unit_min_,synthesis = \
            CalDitorAndCombin(s1_ascending,s1_descending,
                    Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'])

    K = [1 if Method == 'SNIC_Kmean' else 0 for Method in Methods ]

    for k,Method in zip(K,Methods):
        resultband = resultbands[k]
        for Band in Bands:
            if Method in ['Kmean','SNIC_Kmean','LVQ','Xmeans','CascadeKMeans']:
              Map,result = Cluster_math(method=Method,img=s1_unit_min_,bands=Band,index=''
                              ,visual=False,save=False,region=AOI_buffer,
                            proj=volumetric_dict['ASCENDING_parms']['proj'],Synth=synthesis)
              # print('Method 1 {} Band={}'.format(Method,Band))
            elif Method in ['otsu','histPeak']:
              if len(Band)==1:
                Map,result = Bandmath(method=Method,img=s1_unit_min_,band=Band,index='',
                          visual=False,save=False,region=AOI_buffer,
                          proj=volumetric_dict['ASCENDING_parms']['proj'],Synth=synthesis)
                # print('Method 2 {} Band={}'.format(Method,Band))
              else:
                continue
            else:
              print('Wrong method input!')



            # 分类图转矢量
            Union_ex = Reprocess.image2vector(result,
                        resultband=resultband,
                        GLarea=AOI_area_area,
                        FilterBound=FilterBound)

            if s1_ascending.get('synthesis').getInfo() == 0:
              a_name,a_date,a_nodata = ee.List([s1_ascending.get('system:index'),
                                s1_ascending.date().format('YYYY-MM-dd'),
                                s1_ascending.get('numNodata')]).getInfo()
            else:
                a_name = 'None'; a_date = 'None' ; a_nodata='None'

            if s1_descending.get('synthesis').getInfo() == 0:
                d_name,d_date,d_nodata = ee.List([s1_descending.get('system:index'),
                                s1_descending.date().format('YYYY-MM-dd'),
                                s1_descending.get('numNodata')]).getInfo()
            else:
                d_name = 'None'; d_date = 'None' ; d_nodata='None'

            pd_dict = {'a_name':a_name,'d_name':d_name,
                   'a_date':a_date,'d_date':d_date,
                   'a_nodata':a_nodata,'d_nodata':d_nodata}

            # 导出csv和shp
            save_parms.write_pd(Union_ex,i,AOI_area,mode=mode,Method=Method,Band=Band,WithOrigin=resultband,pd_dict=pd_dict,
                  Area_real=AOI_area_area,logname=logname,shapname=shpname,calIoU=True)


    # 导出图像
    try:
        Geemap_export(fileDirname=f'{i:04d}'+'_'+str(resultband)+'Ascending_flat'+a_name+'.tif',collection=False,image=volumetric_dict['ASCENDING'],region=AOI_buffer,scale=10)
        Geemap_export(fileDirname=f'{i:04d}'+'_'+str(resultband)+'Descending_flat'+d_name+'.tif',collection=False,image=volumetric_dict['DESCENDING'],region=AOI_buffer,scale=10)
        Geemap_export(fileDirname=f'{i:04d}'+'_'+str(resultband)+'s2a_sr_median'+'.tif',collection=False,image=s2_sr_median,region=AOI_buffer,scale=10)
    except:
        dir_name = make_dir(f'{i:04d}')
        # 循环输出像素量超标的图像，(裁剪、挨个导出)
        block_list = cut_geometry(AOI_buffer)
        for each in range(len(block_list)):
            Geemap_export(fileDirname=os.path.join(dir_name,f'{i:04d}'+'_'+str(resultband)+'Ascending_'+a_name+'.tif')
                        ,collection=False,image=volumetric_dict['ASCENDING'],region=block_list[each],scale=10)
            Geemap_export(fileDirname=os.path.join(dir_name,f'{i:04d}'+'_'+str(resultband)+'Descending_'+d_name+'.tif')
                        ,collection=False,image=volumetric_dict['DESCENDING'],region=block_list[each],scale=10)
            Geemap_export(fileDirname=os.path.join(dir_name,f'{i:04d}'+'_'+str(resultband)+'s2a_sr_median'+'.tif')
                        ,collection=False,image=s2_sr_median,region=block_list[each],scale=10)

# 新段落